## Publisher:
Will publish the frames at 720@60 fps to the subscriber (detection and tracking pipeline).

In [1]:
import cv2, time, zmq, json, subprocess, threading

CAM_CONFIGS = {
    "kreo1": {
        "dev": 4,
        "json": "../config/camera_tune_results/best_camera_settings_kreo1.json",
        "topic": b"kreo1"
    },
    "kreo2": {
        "dev": 2,
        "json": "../config/camera_tune_results/best_camera_settings_kreo2.json",
        "topic": b"kreo2"
    }
}

def apply_settings(json_path, dev_idx):
    """Applies v4l2 settings before starting the stream."""
    try:
        with open(json_path, "r") as f:
            cfg = json.load(f)

        cmds = [
            ("exposure_time_absolute", cfg.get("exp")),
            ("gain", cfg.get("gain")),
            ("focus_absolute", cfg.get("focus")),
            ("brightness", cfg.get("brightness")),
        ]

        for ctrl, val in cmds:
            if val is None: continue
            subprocess.run(["v4l2-ctl", "-d", f"/dev/video{dev_idx}", "-c", f"{ctrl}={int(val)}"],
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"[Settings] Applied to /dev/video{dev_idx}")
    except Exception as e:
        print(f"[Settings] Error applying {json_path}: {e}")


class CameraWorker(threading.Thread):
    def __init__(self, name, dev, topic, pub, lock, jpeg_quality=80):
        super().__init__(daemon=True)
        self.name = name
        self.dev = dev
        self.topic = topic
        self.pub = pub
        self.lock = lock
        self.running = True
        self.jpeg_quality = jpeg_quality

        self.cap = cv2.VideoCapture(dev, cv2.CAP_V4L2)
        self.cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        self.cap.set(cv2.CAP_PROP_FPS, 60)
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

        if not self.cap.isOpened():
            print(f"[ERR] Cannot open {name} /dev/video{dev}")

    def run(self):
        print(f"[{self.name}] Camera thread started.")
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                continue
            
            ts = time.time()
            
            # Encode to JPEG
            ok, jpg = cv2.imencode(".jpg", frame, [cv2.IMWRITE_JPEG_QUALITY, self.jpeg_quality])
            if not ok:
                continue

            # CRITICAL FIX: Use lock when sending
            with self.lock:
                self.pub.send_multipart([self.topic, str(ts).encode("ascii"), jpg.tobytes()])
                
        try:
            self.cap.release()
        except: pass

    def stop(self):
        self.running = False


def main():
    # ZeroMQ PUB socket
    ctx = zmq.Context()
    pub = ctx.socket(zmq.PUB)
    pub.bind("tcp://*:5555")
    pub.setsockopt(zmq.SNDHWM, 2)
    # pub.setsockopt(zmq.IMMEDIATE, 1)
    pub.setsockopt(zmq.LINGER, 0)

    zmq_lock = threading.Lock()
    # --- ACTIVE FLUSH FOR PUB ---
    try:
        pub.send_multipart([b"FLUSH", b"0", b"0"], flags=zmq.NOBLOCK)
    except zmq.Again:
        pass
    time.sleep(1)
    print("[Publisher] ZMQ buffers flushed.")
    # Apply settings + start workers
    workers = []
    for name, cfg in CAM_CONFIGS.items():
        apply_settings(cfg["json"], cfg["dev"])
        w = CameraWorker(name, cfg["dev"], cfg["topic"], pub, zmq_lock)
        w.start()
        workers.append(w)

    print("[Publisher] Running… Ctrl+C to exit.")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        pass
    finally:
        print("Stopping....")
        for w in workers: w.stop()
        for w in workers: w.join()
        pub.close()
        ctx.term()


if __name__ == "__main__":
    main()


[Publisher] ZMQ buffers flushed.
[Settings] Applied to /dev/video4
[kreo1] Camera thread started.
[Settings] Applied to /dev/video2
[kreo2] Camera thread started.
[Publisher] Running… Ctrl+C to exit.
Stopping....
